In [1]:
import os.path

import numpy as np
import pandas as pd
from scipy import linalg
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder

import Bio.PDB, warnings
pdb_list = Bio.PDB.PDBList()
pdb_parser = Bio.PDB.PDBParser()
from scipy.spatial import distance_matrix
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

from joblib import Parallel, delayed
import timeit

import matplotlib.pyplot as plt
%matplotlib inline

# # --- Import our Code ---# #
#import emachine as EM
from direct_info import direct_info

# import data processing and general DCA_ER tools
from data_processing import data_processing
import ecc_tools as tools
from pathlib import Path
np.random.seed(1)


def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

## Read in Protein Structure
1. We will start by reading in a set of Protein structures with the given Protein ID "pfam_id"
    * [More Info on working with PDB structures in DataFrames](http://rasbt.github.io/biopandas/tutorials/Working_with_PDB_Structures_in_DataFrames/)

In [2]:
data_path = Path('/home', 'eclay','Pfam-A.full')
data_path = Path('Pfam-A.full')
DCA_ER_dir = Path('/home/evan/PycharmProjects/DCA_ER/')



pfam_id = 'PF00023'
pfam_id = 'PF00011'
pfam_id = 'PF00186'
pfam_id = 'PF14525'
pfam_id = 'PF03496'


n_jobs = 6
create_new = True


if 0:
    DCA_ER_dir = '/home/eclay/DCA_ER/'
    msa_npy_file = '/home/eclay/Pfam-A.full/%s/msa.npy' % pfam_id # Hurricane Location
    msa_fa_file  = '/home/eclay/Pfam-A.full/%s/msa.fa' % pfam_id # Hurricane Location
    pdb_ref_file = '/home/eclay/Pfam-A.full/%s/pdb_refs.npy' % pfam_id # Hurricane Location
if 1:
    DCA_ER_dir = '/home/evan/PycharmProjects/DCA_ER/'
    msa_npy_file = '/home/evan/PycharmProjects/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/home/evan/PycharmProjects/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/home/evan/PycharmProjects/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir
if 0:
    DCA_ER_dir = '/home/ecresswell/DCA_ER/'
    msa_npy_file = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/home/ecresswell/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir
if 1:
    DCA_ER_dir = '/data/cresswellclayec/DCA_ER/'
    msa_npy_file = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/msa.npy' % pfam_id
    msa_fa_file  = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/msa.fa' % pfam_id
    pdb_ref_file = '/data/cresswellclayec/DCA_ER/Pfam-A.full/%s/pdb_refs.npy' % pfam_id
    out_dir = '%sprotein_data/di/' % DCA_ER_dir




# Set DCA_ER directory
DCA_dir = os.getcwd()

# Define data directories
# Need to think on best way to do this..
# Referencing the same dataframe may be useful so we dont always have to load individual ref files...
# however we also
individual_pdb_ref_file = Path(data_path, pfam_id, 'pdb_refs.npy')
pdb = np.load(individual_pdb_ref_file)
processed_data_dir = "%s/protein_data/data_processing_output" % DCA_dir

# delete 'b' in front of letters (python 2 --> python 3)
pdb = np.array([pdb[t,i].decode('UTF-8') for t in range(pdb.shape[0]) \
         for i in range(pdb.shape[1])]).reshape(pdb.shape[0],pdb.shape[1])


# Print number of pdb structures in Protein ID folder
npdb = pdb.shape[0]
print('number of pdb structures:',npdb)

# Create pandas dataframe for protein structure
pdb_df = pd.DataFrame(pdb,columns = ['PF','seq','id','uniprot_start','uniprot_end',\
                                 'pdb_id','chain','pdb_start','pdb_end'])
pdb_df.head()

# print("Direct Information from Expectation reflection:\n",di)
def no_diag(mat, diag_l, s_index=None, make_big=False):
    rows, columns = mat.shape
    if make_big:
        new_mat = 100. * np.ones((rows,columns))
    else:
        new_mat = np.zeros((rows,columns))
    for row in range(rows):
        for col in range(columns):
            if s_index is None:
                if abs(row-col) > diag_l:
                    new_mat[row, col] = mat[row ,col]
            else:
                if abs(s_index[row]-s_index[col]) > diag_l:
                    new_mat[row, col] = mat[row ,col]    
    return new_mat


number of pdb structures: 2


PDB DataFrame needs to have PF######, PDB-id, as well as start and end locations of amino acids, as well as reference sequence in msa (not in current file with all pdb data)
### PDB Website Search Engine
   * We can visualize the complex structure of the proteins
   * The data-base below gives further information on the proteins above (input pdb_id)

### Define contact map
2. Import contact map functions from newly defined tools python module

In [3]:
import ecc_tools as tools

In [4]:
ipdb = 0
printing = True
print('seq:',int(pdb[ipdb,1]))



s0,cols_removed, s_index, tpdb, orig_seq_len = data_processing(data_path, pfam_id, ipdb,\
                gap_seqs=0.2, gap_cols=0.2, prob_low=0.004, conserved_cols=0.9, printing=printing, out_dir=processed_data_dir)
print('\n\n\nPreprocessed reference Sequence: ', s0[tpdb])

# npy2fa does not remove cols this way we are as close to original as possible
msa_outfile, ref_outfile, s0, cols_removed, s_index, tpdb, orig_seq_len  = tools.npy2fa(pfam_id, msa_npy_file, pdb_ref_file=pdb_ref_file, ipdb=ipdb, preprocess=True,gap_seqs=.2, gap_cols=.2, prob_low=.004, conserved_cols=.9, letter_format=False)
# data processing

print(s0.shape, '\n\n\n')

# # Load pydca trimmed datrimmed_data_outfile = '%sprotein_data/data_processing_output/MSA_%s_Trimmed.fa' % (DCA_ER_dir, pfam_id)
# trimmed_msa_file = Path(DCA_ER_dir, 'protein_data/data_processing_output/MSA_%s_Trimmed.fa' % pfam_id)
# s0_pydca = tools.read_FASTA(str(trimmed_msa_file), ref_index = int(pdb[ipdb,1]))

# Load csv of PDB-PFAM mapping.
#    downloaded from 
#    ftp://ftp.ebi.ac.uk/pub/databases/msd/sifts/flatfiles/csv/pdb_pfam_mapping.csv.gz
pdb_id = pdb_df.iloc[ipdb]['pdb_id']
pdb_chain = pdb_df.iloc[ipdb]['chain']

pdb_pfam_map_file = Path('%s/protein_data/pdb_data/pdb_pfam_mapping.csv' % DCA_dir)
pdb_map_df = pd.read_csv(pdb_pfam_map_file, sep=',', header=1)
print(pdb_map_df.head())

pdb_id_map_df = pdb_map_df.loc[pdb_map_df['PDB']==pdb_id.lower()]
pdb_pfam_map = pdb_id_map_df.loc[pdb_id_map_df['CHAIN']==pdb_chain]

# print('wtf pdb length matching S sequence length ', orig_seq_len )
# for pdb_index, pdb_info in pdb_pfam_map.iterrows(): # loop trhough rows of dataframe
#     pdb_len = pdb_info['PDB_END'] - pdb_info['PDB_START']
#     print(pdb_len)
#     if pdb_len == orig_seq_len:
#         break
        
#       

seq: 445
Original Sequence length:  912
pdb ref example (pdb[0])  (after UTF-8 decode, removing 'b'):
 ['PF03496' '445' 'Q88A91_PSESM' '108' '258' '3U0J' 'B' '108' '258']
tpdb (s_ipdb) is :  445
#

-------------------------Remove Gaps--------------------------#
Shape of s is :  (469, 912)
s = 
 [['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ...
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']
 ['-' '-' '-' ... '-' '-' '-']]
s[tpdb] shape is  (151,)
s = 
 [['s' 't' 'l' ... 'i' 's' 'l']
 ['-' '-' '-' ... '-' '-' '-']
 ['m' 'v' 'i' ... 'g' 'v' 'i']
 ...
 ['i' 'n' 'n' ... 'd' 'g' 'm']
 ['h' 'g' 'l' ... 'p' 'd' 'p']
 ['t' 'y' 'i' ... 'k' 't' 'l']]
though s still has gaps, s[445] does not:
 ['t' 'p' 'l' 'Y' 'R' 'E' 'V' 'N' 'N' 'Y' 'L' 'r' 'l' 'q' 'H' 'E' 'N' 'S'
 'G' 'R' 'E' 'A' 'E' 'I' 'D' 'N' 'h' 'd' 'E' 'K' 'L' 'S' 'P' 'H' 'I' 'K'
 'M' 'L' 'S' 'S' 'A' 'L' 'N' 'R' 'L' 'm' 'd' 'V' 'A' 'A' 'F' 'R' 'G' 'T'
 'V' 'Y' 'R' 'G' 'I' 'R' 'G

In [5]:
## Generate and Plot Contact Map from PDB coordinates!
# Check that pdb--pfam mapping is unique
if pdb_pfam_map.shape[0] > 1:
    print('Unable to get unique PDB-->Pfam mapping')
    print(pdb_pfam_map)

# pp_range = [pdb_info['PDB_START'], pdb_info['PDB_END']]
pp_range = [pdb_pfam_map.iloc[0]['PDB_START'], pdb_pfam_map.iloc[0]['PDB_END']]
print('Polypeptide range for contact map: ', pp_range)


pdb_out = "%s/protein_data/pdb_data" % DCA_dir
# Directory for storing PDB data locally
# returns contact map with the appropriate columns removed..
# For list of retained columns us s_index
ct, ct_full, n_amino_full, poly_seq_curated = tools.contact_map(pdb, ipdb, pp_range, cols_removed, s_index, pdb_out_dir=pdb_out, printing=printing)
print(ct.shape)
print(ct_full.shape)


ct_mat = ct
# tools.hide_toggle()
print('Contact matrix shape:', ct_mat.shape)

plt.tight_layout(h_pad=1, w_pad=1.5)
plt.title('Contact Matrix')
plt.imshow(ct_mat,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0,25)
plt.colorbar(fraction=0.045, pad=0.05)
# 'I'' 'Q' 'S' 'R' 'V' 'E' 'T' 'V' 'P' 'Y' 'L' 'F' 'C' 'R' 'S' 'E' 'E' 'V'
#  'R' 'P' 'A' 'G' 'M' 'V' 'W' 'Y' 'S' 'I' 'L' 'K' 'D' 'T' 'K' 'I' 'T' 'C'
#  'E' 'E' 'K' 'M' 'V' 'S' 'M' 'A' 'R' 'N' 'T


Polypeptide range for contact map:  [114, 264]


#-----------------------#
Generating Contact Map
#----------------------------#

['PF03496' '445' 'Q88A91_PSESM' '108' '258' '3U0J' 'B' '108' '258']
peptide build of chain produced 1 elements
original poly_seq: 
 ['E', 'R', 'H', 'Y', 'S', 'T', 'G', 'Q', 'D', 'R', 'H', 'D', 'F', 'Y', 'R', 'F', 'A', 'A', 'R', 'L', 'H', 'V', 'D', 'A', 'Q', 'C', 'F', 'G', 'L', 'S', 'I', 'D', 'D', 'L', 'M', 'D', 'K', 'F', 'S', 'D', 'K', 'H', 'F', 'R', 'A', 'E', 'H', 'P', 'E', 'Y', 'R', 'D', 'V', 'Y', 'P', 'E', 'E', 'C', 'S', 'A', 'I', 'Y', 'M', 'H', 'T', 'A', 'Q', 'D', 'Y', 'S', 'S', 'H', 'L', 'V', 'R', 'G', 'E', 'I', 'G', 'T', 'P', 'L', 'Y', 'R', 'E', 'V', 'N', 'N', 'Y', 'L', 'R', 'L', 'Q', 'H', 'E', 'N', 'S', 'G', 'R', 'E', 'A', 'E', 'I', 'D', 'N', 'H', 'D', 'E', 'K', 'L', 'S', 'P', 'H', 'I', 'K', 'M', 'L', 'S', 'S', 'A', 'L', 'N', 'R', 'L', 'M', 'D', 'V', 'A', 'A', 'F', 'R', 'G', 'T', 'V', 'Y', 'R', 'G', 'I', 'R', 'G', 'D', 'L', 'D', 'T', 'I', 'A', 'R', 'L

IndexError: index 142 is out of bounds for axis 0 with size 123

In [ ]:
## Plot the contact Map of the imported Protein structure

## Now we want to use Expectation Reflection to infer the "calculate the weights " 
  * Infer interactions between residues (R-part of ammino acid) in proteins from Multiple Sequence Alignment (MSA) data
  * More understanding required on meaning/interpretation of weights
  
1. We can start by defining some important values of our protein structures
    * It is important to define 'i1i2' vector to keep track of OneHot index positions (i) coresonding to the beginning and end of a given amino acid position (this acts as a key between the OneHot matrix (s) and the original sequence matrix (s0)

In [6]:
# number of positions
n_var = s0.shape[1]
n_seq = s0.shape[0]

print("Number of residue positions:",n_var)
print("Number of sequences:",n_seq)

# number of aminoacids at each position
mx = np.array([len(np.unique(s0[:,i])) for i in range(n_var)])
#mx = np.array([m for i in range(n_var)])
print("Number of different amino acids at each position",mx)

mx_cumsum = np.insert(mx.cumsum(),0,0)
i1i2 = np.stack([mx_cumsum[:-1],mx_cumsum[1:]]).T 
# print("(Sanity Check) Column indices of first and (",i1i2[0],") and last (",i1i2[-1],") positions")
# print("(Sanity Check) Column indices of second and (",i1i2[1],") and second to last (",i1i2[-2],") positions")


# number of variables
mx_sum = mx.sum()
print("Total number of variables",mx_sum)

# number of bias term
n_linear = mx_sum - n_var

Number of residue positions: 151
Number of sequences: 367
Number of different amino acids at each position [ 2  2  2 11 19 19 10  2 18 19  8  1  1  1 18 17 19 19  2  2  2  2  2 19
  2  2  2  1 19 19 17 20 18 19 14 18 20  8 12 14 12  7 18 18 20  2  2 16
 16 17 17 16 17 19 12 18  2 16 16  2  2  2  2  2 19 20 19 19 20 20 19  2
 19 20 15 11 18 18 16 18 16 17 13  8 11  2  8  6 15 19 18 18  2  2 19 20
 18 20 20 19  2  2  2  2 20 21 16 20  9 20 15 19 17 15 17 19 13 17 18 17
 16 19 20 19 20  4 17  2  9  4  8 15 17 13 16 19  9 18  9 14 12 13  2  2
  2  2  2  2  2  2  2]
Total number of variables 1829


2.  The above indices help to divide all the possible amino acids at different positions to binary choices which will be implemented wit 1 of K notation (OneHot Encoder)

In [7]:
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
# s is OneHot encoder format, s0 is original sequnce matrix
s = onehot_encoder.fit_transform(s0)
# print("Amino Acid sequence Matrix\n",s0)
# print("OneHot sequence Matrix\n",s)
# print("An individual element of the OneHot sequence Matrix (size:",
#      s.shape,") --> ",s[0], " has length ",s[0].shape)

#### OneHot Format example
* Given 3 sequence poitions each with 3 different amino acids:
    * Notation: The 3<sup>rd</sup> amino acid type at the 2<sup>nd</sup> sequence position is denoted as R23<br/>
* OneHot converts the sequence [R11 R22 R33] &rightarrow; [1 0 0 0 1 0 0 0 1]  
  
  
3. We next want to define our local field parameters

In [8]:
# Define wight matrix with variable for each possible amino acid at each sequence position
w = np.zeros((mx.sum(),mx.sum())) 
h0 = np.zeros(mx.sum())

### Apply Expectation Reflection
4. We can now apply expectation reflection to infer the connection between postion residues

In [9]:
import sys
import numpy as np
from scipy import linalg
from sklearn.preprocessing import OneHotEncoder
import expectation_reflection as ER
from direct_info import direct_info
from joblib import Parallel, delayed

In [10]:
# Expectation Reflection
#=========================================================================================
def predict_w(s,i0,i1i2,niter_max,l2):
    #print('i0:',i0)
    i1,i2 = i1i2[i0,0],i1i2[i0,1]

    x = np.hstack([s[:,:i1],s[:,i2:]])
    y = s[:,i1:i2]

    h01,w1 = ER.fit(x,y,niter_max,l2)

    return h01,w1


if os.path.exists("%s_w.npy" % pfam_id) and not create_new:
# if 0:
    w = np.load("%s_w.npy" % pfam_id)
else:
    #-------------------------------
    # parallel
    start_time = timeit.default_timer()
    #res = Parallel(n_jobs = 4)(delayed(predict_w)\
    res = Parallel(n_jobs = n_jobs)(delayed(predict_w)\
    #res = Parallel(n_jobs = 32)(delayed(predict_w)\
            (s,i0,i1i2,niter_max=10,l2=100.0)\
            for i0 in range(n_var))

    run_time = timeit.default_timer() - start_time
    print('run time:',run_time)
    ## This above line seems wrong, seems like the following for loop should be moved up?? not sure if this is some 
    ## python implementation or just wrong
    #----------------niter_max,l2)

    #-------------------------------
    # parallel
    start_time = timeit.default_timer()
    #res = Parallel(n_jobs = 4)(delayed(predict_w)\
    #res = Parallel(n_jobs = 8)(delayed(predict_w)\
    res = Parallel(n_jobs = 32)(delayed(predict_w)\
            (s,i0,i1i2,niter_max=10,l2=100.0)\
            for i0 in range(n_var))

    run_time = timeit.default_timer() - start_time
    print('run time:',run_time)
    ## This above line seems wrong, seems like the following for loop should be moved up?? not sure if this is some 
    ## python implementation or just wrong
    #-------------------------------
    for i0 in range(n_var):
        i1,i2 = i1i2[i0,0],i1i2[i0,1]

        h01 = res[i0][0]
        w1 = res[i0][1]

        h0[i1:i2] = h01
        w[:i1,i1:i2] = w1[:i1,:]
        w[i2:,i1:i2] = w1[i1:,:]

    # make w symmetric
    w = (w + w.T)/2.



run time: 117.73817592859268
run time: 67.10801172628999


5. Now we want to plot a heat map of our sequence weights

In [11]:
# Verify that w is symmetric (sanity test)
print("Dimensions of w: ",w.shape)
w_file = "%s_w.npy" % pfam_id
np.save(w_file, w)

Dimensions of w:  (1829, 1829)


6. Plot relationship between sequence positions via Direct information

In [12]:
di = direct_info(s0,w)
print(di)
print(di.shape)
print(len(s_index))

from ecc_tools import score_APC
sequence_length = pp_range[1] - pp_range[0]

# RunE ER di scores through Average Product Correction (to combat influences of phylogeny)
er_APC = score_APC(di, sequence_length, s_index)
di_apc = tools.di_dict2mat(er_APC, s_index, cols_removed = cols_removed)

[[0.         0.00028071 0.00028072 ... 0.00028081 0.00028073 0.00028072]
 [0.00028071 0.         0.0002807  ... 0.00028079 0.00028071 0.0002807 ]
 [0.00028072 0.0002807  0.         ... 0.0002808  0.00028071 0.00028071]
 ...
 [0.00028081 0.00028079 0.0002808  ... 0.         0.0002808  0.0002808 ]
 [0.00028073 0.00028071 0.00028071 ... 0.0002808  0.         0.00028071]
 [0.00028072 0.0002807  0.00028071 ... 0.0002808  0.00028071 0.        ]]
(151, 151)
151
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 1

# Plotting ER and DCA
### Plot ER

In [ ]:
#print("Direct Information from Expectation reflection:\n",di)
print('ER DI shape: ' , di.shape)
ER_di = np.delete(di, cols_removed,0)
ER_di = np.delete(ER_di, cols_removed,1)

print('Final ER DI shape (cols removed): ', ER_di.shape)

ER_di_nodiag = no_diag(ER_di, 5, s_index)
#print(ER_di_nodiag)
#ER_di = ER_di_nodiag

In [ ]:
plt.title('direct information')
plt.imshow(ER_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0,0.01)
plt.colorbar(fraction=0.045, pad=0.05)

In [ ]:
from ecc_tools import roc_curve
# find optimal threshold of distance
ct_thres = np.linspace(4.,6.,18,endpoint=True)
n = ct_thres.shape[0]


print(ct.shape)
# Initialize plotting
iplot = [1,3,5,7,9,11,13]
plt.figure(figsize=(9.0,3.2))

# Initalize ROC-curve tile
plt.subplot2grid((1,4),(0,0))
plt.title('ROC various thres')
plt.plot([0,1],[0,1],'k--')

# Find ct_threshold (2-6 Angstrom) which gives best contact prediction for ER
auc_ER = np.zeros(n)
for i in range(n):
    try:
        p,tp,fp = roc_curve(ct_mat, ER_di, ct_thres[i])
        if i in iplot:
            plt.plot(fp,tp,label='thres = %3.2f'%ct_thres[i])
        auc_ER[i] = tp.sum()/tp.shape[0]
    except(RuntimeWarning):
        auc_ER[i] = 0

# Get ER method's best contact prediction
i0_ER = np.argmax(auc_ER)
print('ER auc max:',ct_thres[i0_ER],auc_ER[i0_ER])
p0_ER,tp0_ER,fp0_ER = tools.roc_curve(ct_mat,ER_di,ct_thres[i0_ER])

# ROC-curve tile settings
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()


# Plot ROC, AUC and Precision for best ER contact prediction
# AUC
plt.subplot2grid((1,4),(0,1))
plt.title('AUC max = %f' %(auc_ER[i0_ER]))
plt.plot([ct_thres.min(),ct_thres.max()],[0.5,0.5],'k--')
plt.plot(ct_thres,auc_ER,'k-')
plt.plot(ct_thres[i0_ER],auc_ER[i0_ER], 'b*', markersize=10.)
plt.xlim([ct_thres.min(),ct_thres.max()])
plt.ylim([0,auc_ER.max()+0.1])
plt.xlabel('distance threshold')
plt.ylabel('AUC')
# ROC
plt.subplot2grid((1,4),(0,2))
plt.title('ROC at thres = %3.2f'%(ct_thres[i0_ER]))
plt.plot(fp0_ER,tp0_ER,'b-')
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# Precision-Recall
plt.subplot2grid((1,4),(0,3))
plt.title('Precision-Recall Curve')
print('\nContact threshold: ', ct_thres[i0_ER])
plt.plot( p0_ER,tp0_ER / (tp0_ER + fp0_ER), 'b-', label='thres = %s'%ct_thres[i0_ER])
plt.xlabel('Recall (Sensitivity - P)')
plt.ylabel('Precision (PPV)')



### Plot DCA 

In [ ]:
from inference_dca import direct_info_dca
print(s0.shape)
seq_wt_file = None
seq_wt_file = '%s/protein_data/data_processing_output/seq_weight_%s.npy' % (DCA_ER_dir, pfam_id)

# ----------- DCA DI (MF) calculation --------------------------------------------- #

mf_di, fi, fij, c, cinv, w, w2d, fi_pydca, fij_pydca, c_pydca, c_inv_pydca, \
w_pydca, w2d_pydcak, di_pydca, ma_inv,seq_ints\
= direct_info_dca(s0, seq_wt_outfile=seq_wt_file)
print('c[0]:',c[0])
print('c_pydca[0]', c_pydca[0])

In [ ]:
MF_di = np.delete(mf_di, cols_removed,0)
MF_di = np.delete(MF_di, cols_removed,1)
MF_hybrid_di = np.delete(di_pydca, cols_removed,0)
MF_hybrid_di = np.delete(MF_hybrid_di, cols_removed,1)
# --------------------------------------------------------------------------------- #
print(MF_di.shape)
plt.title('Tai MF DI')
plt.imshow(MF_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0.,.01)
plt.colorbar(fraction=0.045, pad=0.05)

plt.figure()
print(MF_hybrid_di.shape)
plt.title('Hybrid MF DI')
plt.imshow(MF_hybrid_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(2.,0.)
plt.colorbar(fraction=0.045, pad=0.05)
print('cinv[0]', cinv[0])

In [ ]:
# Run our version of MF di scores through Average Product Correction (to combat influences of phylogeny)
# mf_APC = score_APC(MF_di, sequence_length, s_index)
# MF_di = tools.di_dict2mat(mf_APC, cols_removed, s_index)

# Initialize plotting
iplot = [1,3,5,7,9,11,13]
plt.figure(figsize=(9.0,3.2))

plt.subplot2grid((1,4),(0,0))
plt.title('ROC various thres')
plt.plot([0,1],[0,1],'k--')

# find optimal threshold of distance
auc_MF = np.zeros(n)

for i in range(n):
    try:
        p,tp,fp = roc_curve(ct_mat, MF_di, ct_thres[i])
        if i in iplot:
            plt.plot(fp,tp,label='thres = %3.2f'%ct_thres[i])
        auc_MF[i] = tp.sum()/tp.shape[0]
    except(RuntimeWarning):
        auc_MF[i] = 0
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
        
i0_MF = np.argmax(auc_MF)
print('auc max:',ct_thres[i0_MF],auc_MF[i0_MF])
p0_MF,tp0_MF,fp0_MF = tools.roc_curve(ct,MF_di,ct_thres[i0_MF])


plt.subplot2grid((1,4),(0,1))
plt.title('AUC max = %f' %(auc_MF[i0_MF]))
plt.plot([ct_thres.min(),ct_thres.max()],[0.5,0.5],'k--')
plt.plot(ct_thres,auc_MF,'r-')
plt.plot(ct_thres[i0_MF],auc_MF[i0_MF], 'r*', markersize=10.)
plt.xlim([ct_thres.min(),ct_thres.max()])
plt.ylim([0,auc_MF.max()+0.1])
plt.xlabel('distance threshold')
plt.ylabel('AUC')

plt.subplot2grid((1,4),(0,2))
plt.title('ROC at thres = %3.2f'%(ct_thres[i0_MF]))
plt.plot(fp0_MF,tp0_MF,'r-')
plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.subplot2grid((1,4),(0,3))
plt.title('Precision-Recall Curve')
plt.plot(p0_MF, tp0_MF / (tp0_MF + fp0_MF), 'r-', label='thres = %s'%ct_thres[i0_MF])
plt.xlabel('Recall (Sensitivity - P)')
plt.ylabel('Precision (PPV)')

plt.tight_layout(h_pad=1, w_pad=1.5)

# Load PYDCA Results

In [ ]:
def slice_couplings(couplings = None, site_pair = None, num_site_states=None):
    """Returns couplings corresponding to site pair (i, j). Note that the
    the couplings involving gaps are included, but they are set to zero.

    Parameters
    ----------
        couplings : np.array
            A 2d numpy array of couplings. It has a shape of (L(q-1), L(q-1))
            where L and q are the length of sequences in alignment data and total
            number of standard residues plus gap.
        site_pair : tuple
            A tuple of site pairs. Example (0, 1), (0, L-1), ..., (L-2, L-1).
        num_site_states : int
            The value of q.

    Returns
    -------
        couplings_ij : np.array
            A2d numpy array of shape (q, q) containing the couplings. Note that
            couplings_ij[q, :] and couplings[:, q] are set to zero.
    """
    q = num_site_states
    couplings_ij = np.zeros((q, q), dtype = np.float64)
    row_begin = site_pair[0] * (q - 1)
    row_end = row_begin + q - 1
    column_begin = site_pair[1] * (q -1)
    column_end = column_begin + q - 1
    couplings_ij[:q-1, :q-1] = couplings[row_begin:row_end, column_begin:column_end]
    return couplings_ij

# -------------------- Load PYDCA Data ------------------------------------------------------------------- #
DCA_ER_dir = '/home/evan/temp_DCA_ER_pydcaMF/DCA_ER/'
DCA_ER_dir = '/home/evan/PycharmProjects/DCA_ER/'
DCA_ER_dir = '/home/ecresswell/DCA_ER/'
DCA_ER_dir = '/data/cresswellclayec/DCA_ER/'

out_dir = '%sprotein_data/di/' % DCA_ER_dir
method = "DI"


preprocess_pydca = False
if preprocess_pydca:
    # Load pydca score pseudo-dictionaries and generate di matrices from them..
    mf_score_file = '%s%s_%s_pydca_mf_preproc.npy' % (out_dir, pfam_id, method)
    plm_score_file = '%s%s_%s_pydca_plm_preproc.npy' % (out_dir, pfam_id, method)
else:
    # Load pydca score pseudo-dictionaries and generate di matrices from them..
    mf_score_file = '%s%s_%s_pydca_mf.npy' % (out_dir, pfam_id, method)
    plm_score_file = '%s%s_%s_pydca_plm.npy' % (out_dir, pfam_id, method)  
# list of di scores length of: seq_len choose 2 
# (ie for PF00186 with sequence length 160, this should be length 160 choose 2 = 12720)
mf_scores = np.load(mf_score_file, allow_pickle=True)
plm_scores = np.load(plm_score_file, allow_pickle=True)

# Load pydca data for comparison
reg_fij = np.load('%s%s_pydca_fij_preproc.npy' % (out_dir, pfam_id))
reg_fi = np.load('%s%s_pydca_fi_preproc.npy' % (out_dir, pfam_id))
corr_mat = np.load('%s%s_pydca_corr_preproc.npy' % (out_dir, pfam_id))                                                                             
couplings = np.load('%s%s_pydca_couplings_preproc.npy' % (out_dir, pfam_id))        
fields_ij = np.load('%s%s_pydca_fields_preproc.npy' % (out_dir, pfam_id))

# print(corr_mat[1][:20])
# print(c[1][:20])
# print(corr_mat.shape)
# print(c.shape)
# -------------------------------------------------------------------------------------------------------- #
# plot difference between coupling matrices (PYDCA MF and our MF)

from inference_dca import interactions, direct_info, mapkey
l = s0.shape[1] # l --> number of aa in each sequence
print('aa sequence length l = %d' % l)

q = 21


print(couplings.shape) # 160 pos x 20 aa
print(w2d.shape) # 137 pos x 21 aa
print(cinv.shape)
print(c.shape)
print(w[0][5][0])
print(w.shape)

w_2D = np.zeros(couplings.shape)
w_vals = []
for i in range(l):
    for j in range(i, l):
        for alpha in range(q-1):
            row = mapkey(i,alpha,q)
            for beta in range(q-1):
                col=mapkey(j,beta,q)
                try:
                    w_2D[row, col] = w[i][j][alpha][beta]
                    w_2D[col, row] = w[i][j][alpha][beta]
                    w_vals.append(couplings[row,col])
                except(IndexError):
                    print(i, j, alpha, beta)
                    print(w_2D)
print('min value in w: ', min(w_vals))
print('max value in w: ', max(w_vals))

## Compare Optimal MF and ER
* Taking the optimal threshold for each gives comparable accuracy
* ER Has higher accuracy

In [ ]:


# print(reg_fi.shape)
# print(np.delete(reg_fi, cols_removed, axis=0))
# print(np.delete(reg_fi, cols_removed, axis=0).shape)
# print(fi)
# print(fi.shape)

# print(len(mf_fn_apc))
# print(MF_di.shape)
# print(mf_fn_apc[:5])
pydca_MF_di = tools.di_dict2mat(mf_scores, s_index, cols_removed=cols_removed, aa_index_correction=True)
pydca_PLM_di = tools.di_dict2mat(plm_scores, s_index, cols_removed=cols_removed, aa_index_correction=True)

# PYDCA_MF_di = np.delete(pydca_MF_di, cols_removed,0)
# PYDCA_MF_di = np.delete(PYDCA_MF_di, cols_removed,1)

# PYDCA_PLM_di = np.delete(pydca_PLM_di, cols_removed,0)
# PYDCA_PLM_di = np.delete(PYDCA_PLM_di, cols_removed,1)

print('\n',pydca_PLM_di.shape)
print(pydca_MF_di.shape)

print(max(s_index))

plt.title('PYDCA MF direct information')
plt.imshow(pydca_MF_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0.,.01)
plt.colorbar(fraction=0.045, pad=0.05)
plt.figure()
plt.title('PYDCA PLM direct information')
plt.imshow(pydca_PLM_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0.,.01)
plt.colorbar(fraction=0.045, pad=0.05)

In [ ]:
non_triv_cinv = np.nonzero(cinv)
print(non_triv_cinv)
non_triv_c = np.nonzero(c)
print(non_triv_c)
eig_vals, eig_vecs = np.linalg.eig(c)
hybrid_eig_vals, hybrid_eig_vecs = np.linalg.eig(c_pydca)
pydca_eig_vals, pydca_eig_vegs = np.linalg.eig(corr_mat)
eig_diff = [abs(eig_vals[i] - pydca_eig_vals[i]) for i in range(len(eig_vals))]
print('max, mean, std dev eig diff: ', max(eig_diff), np.mean(eig_diff), np.std(eig_diff))
count = 0
for diff in eig_diff:
    if diff > .001:
        count = count + 1
print('%d eigenvalues with diff greater than .001' % count)
print('Tais min eig: ', min(eig_vals))
print('Hybrid min eig: ', min(hybrid_eig_vals))
print('PYDCAs min eig: ', min(pydca_eig_vals))

In [ ]:
print (len(eig_diff))
plt.figure(figsize=(15.0,12))

# Compare Correlations
plt.subplot2grid((3,3),(0,0))

plt.plot([0,1],[0,1],'k--')
plt.title('Tai correlation\n(max: %f, min: %f)' % (np.amax(c), np.amin(c)))
plt.imshow(c,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(-.0007,.001)
plt.colorbar(fraction=0.045, pad=0.05)

plt.subplot2grid((3,3),(0,1))
plt.title('PYDCA correlation\n(max: %f, min: %f)' % (np.amax(corr_mat), np.amin(corr_mat)))
plt.imshow(corr_mat,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(-.0007,.001)
plt.colorbar(fraction=0.045, pad=0.05)   

plt.subplot2grid((3,3),(0,2))
plt.title('Correlation Diff')
plt.imshow(abs(corr_mat - c),cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0,.001)
plt.colorbar(fraction=0.045, pad=0.05)


# Compare Weights
plt.subplot2grid((3,3),(1,0))

plt.plot([0,1],[0,1],'k--')
plt.title('Tai weights')
plt.imshow(w_2D,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(-.1,1.)
plt.colorbar(fraction=0.045, pad=0.05)

plt.subplot2grid((3,3),(1,1))
plt.title('PYDCA weights')
plt.imshow(couplings,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(-.1,1.)
plt.colorbar(fraction=0.045, pad=0.05)   

plt.subplot2grid((3,3),(1,2))
plt.title('Diff')
plt.imshow(abs(couplings - w_2D),cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0,.2)
plt.colorbar(fraction=0.045, pad=0.05)


# Compare DIs 
plt.subplot2grid((3,3),(2,0))

plt.plot([0,1],[0,1],'k--')
plt.title('Tai MF DI')
plt.imshow(MF_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0.,.01)
plt.colorbar(fraction=0.045, pad=0.05)

plt.subplot2grid((3,3),(2,1))
plt.title('PYDCA MF DI')
plt.imshow(pydca_MF_di,cmap='rainbow',origin='lower')
plt.xlabel('i')
plt.ylabel('j')
plt.clim(0,.01)
plt.colorbar(fraction=0.045, pad=0.05)   
plt.savefig('taiVSpydca_colorplot.pdf')

# ----- PYDCA calculate 'fields' ----- #

# ------------------------------------ #

# # -------------------------- PYDCA's direct info computation --------------------------------------- #
num_unique_pairs = l * (l -1)                                                      
num_unique_pairs /= 2                                                                            
two_site_model_fields = np.zeros((np.int64(num_unique_pairs), 2, q), dtype=np.float64)           
TOLERANCE = 1.0e-4                                                                            
pair_counter = 0


fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(121)
ax.set_title('w dist (full range)')
ax.hist(w_2D.ravel(), alpha=0.5, bins=50, color= 'b', label='Tai weights')
ax.hist(couplings.ravel(), alpha=0.5, bins=50,  color= 'r', label='PYDCA weights')
ax = fig.add_subplot(122)
ax.set_title('w dist (tele range)')
ax.hist(w_2D.ravel(), alpha=0.5,range=(-2.5,5), bins=50, color= 'b', label='Tai weights')
ax.hist(couplings.ravel(), alpha=0.5, bins=50, range=(-2.5,5),  color= 'r', label='PYDCA weights')
ax.legend(loc='upper right', prop={'size':14})
plt.savefig('w_val_distribution.pdf')
# print('MF_di[0]:', MF_di[0, :10])
# print('hybrid_di[0]:', hybrid_di[0, :10])

In [ ]:
# ER_di = np.delete(di, cols_removed,0)
# ER_di = np.delete(ER_di, cols_removed,1)
# Get best auc for PYDCA mf and plm (and plot the ROC curves)

# which contact matrix do you want to use (full or curated)
ct_mat = ct_full # full pdb contact matrix without columns removed
ct_mat = ct  # full pdb contact matrix with conserved and bad columns removed (defined during pre-processing)
plt.figure(figsize=(9.0,3.2))

plt.title('ROC for PYDCA scores')

# PYDCA-MF
auc_pydca_MF = np.zeros(n)

ax1 = plt.subplot2grid((1,2),(0,0))
for i in range(n):
    try:
        p,tp,fp = roc_curve(ct_mat, pydca_MF_di, ct_thres[i])
        if i in iplot:
            ax1.plot(fp,tp,label='thres = %3.2f'%ct_thres[i])
        auc_pydca_MF[i] = tp.sum()/tp.shape[0]
    except(RuntimeWarning):
        auc_MF[i] = 0
ax1.set_xlim([0,1])
ax1.set_ylim([0,1])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('PYDCA MF: %s' % method)
        
i0_pydca_MF = np.argmax(auc_pydca_MF)
print('auc max:',ct_thres[i0_pydca_MF],auc_pydca_MF[i0_pydca_MF])
p0_pydca_MF,tp0_pydca_MF,fp0_pydca_MF = tools.roc_curve(ct_mat,pydca_MF_di,ct_thres[i0_pydca_MF])


# PYDCA-PLM
auc_pydca_PLM = np.zeros(n)

ax2 = plt.subplot2grid((1,2),(0,1))
for i in range(n):
    try:
        p,tp,fp = roc_curve(ct_mat, pydca_PLM_di, ct_thres[i])
        if i in iplot:
            ax2.plot(fp,tp,label='thres = %3.2f'%ct_thres[i])
        auc_pydca_PLM[i] = tp.sum()/tp.shape[0]
    except(RuntimeWarning):
        auc_pydca_PLM[i] = 0
ax2.set_xlim([0,1])
ax2.set_ylim([0,1])
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('PYDCA PLM: %s' %method)
        
i0_pydca_PLM = np.argmax(auc_pydca_PLM)
print('auc max:',ct_thres[i0_pydca_PLM],auc_pydca_PLM[i0_pydca_PLM])
p0_pydca_PLM,tp0_pydca_PLM,fp0_pydca_PLM = tools.roc_curve(ct_mat,pydca_PLM_di,ct_thres[i0_pydca_PLM])

In [ ]:
print('ER auc max:',ct_thres[i0_ER],auc_ER[i0_ER])
print('DCA auc max:',ct_thres[i0_MF],auc_MF[i0_MF])

print('PYDCA MF auc max:',ct_thres[i0_pydca_MF],auc_pydca_MF[i0_pydca_MF])
print('PYDCA PLM auc max:',ct_thres[i0_pydca_PLM],auc_pydca_PLM[i0_pydca_PLM])

In [ ]:
plt.figure(figsize=(12.0,3.2))

# Plot ROC for optimal DCA vs optimal ER
plt.subplot2grid((1,3),(0,0))
plt.title('ROC at thres (MF, ER)\n = (%3.2f, %3.2f)'%(ct_thres[i0_MF],ct_thres[i0_ER]))
plt.plot(fp0_ER,tp0_ER,'b-',label="ER")
plt.plot(fp0_MF,tp0_MF,'r-',label="MF")
plt.plot(fp0_pydca_MF,tp0_pydca_MF,'ro',label="PYDCA MF", markersize=1)
plt.plot(fp0_pydca_PLM,tp0_pydca_PLM,'go',label="PYDCA PLM", markersize=1)

plt.plot([0,1],[0,1],'k--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()

# Plot AUC for DCA and ER
plt.subplot2grid((1,3),(0,1))
plt.title('AUC (MF, ER) max\n = (%3.2f, %3.2f)' %(auc_MF[i0_MF], auc_ER[i0_ER]))
plt.plot([ct_thres.min(),ct_thres.max()],[0.5,0.5],'k--')
plt.plot(ct_thres,auc_ER,'b-',label="ER")
plt.plot(ct_thres,auc_MF,'r-',label="MF")
plt.plot(ct_thres,auc_pydca_MF,'ro',label="PYDCA MF", markersize=1)
plt.plot(ct_thres,auc_pydca_PLM,'go',label="PYDCA PLM", markersize=1)
plt.xlim([ct_thres.min(),ct_thres.max()])
plt.ylim([0,max(auc_MF.max(), auc_ER.max())+0.1])
plt.xlabel('distance threshold')
plt.ylabel('AUC')
plt.legend()

# Plot Precision of optimal DCA and ER
plt.subplot2grid((1,3),(0,2))
plt.title('Precision at thres \n(MF, ER) = (%3.2f, %3.2f)'%(ct_thres[i0_MF],ct_thres[i0_ER]))
plt.plot( p0_ER,tp0_ER / (tp0_ER + fp0_ER),'b-',label='ER thres = %3.2f'%ct_thres[i0_ER])
plt.plot( p0_MF,tp0_MF / (tp0_MF + fp0_MF),'r-',label='MF thres = %3.2f'%ct_thres[i0_MF])
plt.plot( p0_pydca_MF,tp0_pydca_MF / (tp0_pydca_MF + fp0_pydca_MF),'ro',label='PYDCA MF %s thres = %3.2f' % (method, ct_thres[i0_pydca_MF]), markersize=1)
plt.plot( p0_pydca_PLM,tp0_pydca_PLM / (tp0_pydca_PLM + fp0_pydca_PLM),'go',label='PYDCA PLM %s thres = %3.2f' % (method, ct_thres[i0_pydca_PLM]), markersize=1)

plt.plot([0,1.],[.5,.5],'k--')
plt.xlim([0,1])
plt.ylim([0,1]) 
plt.xlabel('Recall (Sensitivity - P)')
plt.ylabel('Precision (PPV)')
plt.legend()

plt.tight_layout(h_pad=1, w_pad=1.5)

plt.savefig('ERvPYDCA_%s_comparison.pdf' % method)


# tools.hide_toggle()